In [1]:

from dotenv import load_dotenv

load_dotenv('../../.env')
import sys
sys.path.insert(0, '../app')
from SIWeaviateClient import SIWeaviateClient
from processing.image.SIImageDescription import SIImageDescription
from processing.text.SIITranslator import SITranslator
from processing.text.SISurya import SISurya
from S3Storage import S3Storage

s3 = S3Storage()
image_descriptor = SIImageDescription()
translator = SITranslator()
surya = SISurya()



/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/pydantic/_internal/_config.py:284: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: DeprecationWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", DeprecationWarning)
/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
 

Loaded detection model vikp/surya_det2 on device cuda with dtype torch.float16
Loaded detection model vikp/surya_layout2 on device cuda with dtype torch.float16
Loaded reading order model vikp/surya_order on device cuda with dtype torch.float16


## process all pdfs

In [ ]:
from weaviate import WeaviateClient
from processing.PDFProcessor import PDFProcessor
import os
from weaviate.classes.query import Filter, GeoCoordinate, MetadataQuery, QueryReference
# from app.SIWeaviateClient import SIWeaviateClient

def get_all_pdfs():
    pdf_files = []
    for root, dirs, files in os.walk("./data"):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
                # pdf_files.append(os.path.abspath(os.path.join(root, file)))
    return pdf_files

def is_document_already_indexed(pdf_path, client: WeaviateClient):
    document = client.collections.get("Document")
    response = document.query.fetch_objects(
        filters=(
            Filter.by_property("original_path").equal(pdf_path)
        ),
        limit=1,
        return_properties=[]
    )
    return len(response.objects) > 0

with SIWeaviateClient() as client:
    print("CLIENT", client)
    pdf_paths = get_all_pdfs()
    for pdf_path in pdf_paths:
        # print("==========================================")
        # print(pdf_path)
        if (is_document_already_indexed(pdf_path, client)):
            print(f"Already in DB, SKIP INDEXING {pdf_path}")
            continue
        # pdf = PDFProcessor(client, image_descriptor, translator, pdf_path)



## process single pdf

In [2]:
from processing.PDFProcessor import PDFProcessor
import os
import glob


pdf_path = "./data/f7170a41-45f9-4e47-b552-9ab05102d7c3.pdf"
with SIWeaviateClient() as client:
    pdf = PDFProcessor(client, image_descriptor, translator, surya, s3, pdf_path)

/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/../app/schemas.py:182: GenericBeforeBaseModelWarning: Classes should inherit from `BaseModel` before generic classes (e.g. `typing.Generic[T]`) for pydantic generics to work properly.
  class ChunkWithScore(Generic[ChunkType], BaseDocumentChunk):


PDFProcessor pdf_path ./data/f7170a41-45f9-4e47-b552-9ab05102d7c3.pdf
PDFProcessor extract_document self.pdf_path ./data/f7170a41-45f9-4e47-b552-9ab05102d7c3.pdf
File ./data/f7170a41-45f9-4e47-b552-9ab05102d7c3.pdf uploaded successfully to bucket science-infuse-content-dev as pdf/b9c2f4f3-57a3-4da7-bfdc-45b5e1e119b4.pdf.
PDFProcessor extract_document images 12


Finding reading order: 100%|██████████| 8/8 [00:04<00:00,  1.69it/s]


pos      : 8
label    : Text
cur_title: 
cur_subtitle: PREAMBULE
text     : Lieu : salle d’animation Nombre d’intervenants : 1 médiateur scientifique (longue préparation préalable) Partenaires : CNRS, Inserm
len text : 131


pos      : 9
label    : Text
cur_title: 
cur_subtitle: PREAMBULE
text     : Objectifs :
- Se repérer dans la terminologie de la biologie et de la génétique.
- Donner des notions d’échelle et de taille.
- Montrer comment les différents constituants du vivant (molécules, organites, cellules...) s’organisent les uns par rapport aux autres.
- Découvrir la molécule d’ADN et son mode d’extraction.
len text : 318


pos      : 11
label    : Text
cur_title: 
cur_subtitle: Scénario :
text     : Le public reconstitue une fresque « de l’organisme à l’atome », avec des images de cellules, de tissus, d’organites, de molécules..., qu’il doit remettre dans l’ordre, en associant la bonne légende à la bonne échelle. Des indices aux dos des images l’aident à se repérer. Dans un deuxi

# test bugs

In [ ]:
import sys
sys.path.insert(0, '../app')
from processing.image.SIImageDescription import SIImageDescription
image_descriptor = SIImageDescription()

In [ ]:
from IPython.display import Image 
from dotenv import load_dotenv
load_dotenv('../../.env')
import sys
sys.path.insert(0, '../app')

import io
import fitz
from PIL import Image
from pymupdf import Document as PdfDocument

def get_pdf_images(doc: type[PdfDocument]):
    temp_images = []

    for page_num in range(len(doc)):
        page = doc[page_num]

        for item in doc.get_page_images(page_num):
            pix = fitz.Pixmap(doc, item[0])  # pixmap from the image xref
            pix0 = fitz.Pixmap(fitz.csRGB, pix)  # force into RGB
            pil_image = Image.frombytes("RGB", [pix0.width, pix0.height], pix0.samples)
            x0, y0, x1, y1 = page.get_image_bbox(item[7])
            width = x1 - x0
            height = y1 - y0
            # print(f"Width: {width}, Height: {height}")
            # print(f"Image bounding box: x0={x0}, y0={y0}, x1={x1}, y1={y1}")
            # display(pil_image)
            temp_images.append({"image": pil_image, "page_number": page_num+1, 'bbox': {"x0": x0, "y0": y0, "x1": x1, "y1": y1, }})

    return temp_images

# image_descriptor = SIImageDescription()
pdf_path = "/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/data/UniversScience/revue-decouverte/decouverte_436.pdf"
# pdf_path = "/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/data/UniversScience/dossiers-pédagogiques/astro/2015/2015 - Dinosaures.pdf"
with fitz.open(pdf_path) as fitz_doc:
    images = get_pdf_images(fitz_doc)
    print(images[0])
    images_chunks = [{**image, "description_en": image_descriptor.get_description(image['image'])} for image in images]

    # for image in images:
    #     display(image)
    # images_chunks = [{**image, "description_en": image_descriptor.get_description(image['image'])} for image in images]


In [ ]:
images_chunks